In [2]:
!pip install ipympl

     |████████████████████████████████| 510 kB 997 kB/s eta 0:00:01


In [3]:
%matplotlib widget

In [4]:
# You need these lines in Google Colab

#from google.colab import output
#output.enable_custom_widget_manager()

In [5]:


import numpy as np
import sympy as sp
import math
import matplotlib.pyplot as plt
import matplotlib.patches as pltp
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import HTML, Image
from sympy.solvers import solve
from sympy import Symbol
from sympy import Ellipse, Point, Line

plt.rcParams['text.usetex'] = False

In [5]:
def solve_task1(scale=1, gif=True):
    fig, ax = plt.subplots()
    ax.set_aspect("equal")
    xdata, ydata = [], []
    ln, = plt.plot([], [], 'b-')
    vecs = []
    caps = []

    def xt(t):
        return 3 * t

    def yt(t):
        return 4 * t**2 + 1

    def yx(x):
        return 4 / 9 * x**2 + 1

    def vt(t):
        return (3, 8 * t)

    def at(t):
        return (0, 8)

    def att(t):
        return np.dot(vt(t), at(t)) / np.sqrt(np.dot(vt(t), vt(t)))

    def ant(t):
        return np.sqrt(np.dot(at(t), at(t)) - att(t)**2)

    def init_yx():
        ax.set_xlim(scale * -50, scale * 50)
        ax.set_ylim(scale * -1, scale * 105)
        plt.xlabel('x')
        plt.ylabel('y(x)')
        plt.grid()
        plt.autoscale(False)
        plt.title('Original Size')
        return ln,

    def update_yx(frame):

        xdata.append(xt(frame))
        ydata.append(yx(xt(frame)))

        while len(vecs):
            vecs[-1].remove()
            vecs.pop()

        while len(caps):
            caps[-1].remove()
            caps.pop()

        scale_factor = 1

        xt_val = xt(frame)
        yt_val = yt(frame)
        yx_val = yx(xt_val)
        att_val = att(frame)
        ant_val = ant(frame)

        vt_vec = vt(frame)
        tan_vec = vt_vec / np.sqrt(np.dot(vt_vec, vt_vec))
        norm_vec_c = complex(tan_vec[0], tan_vec[1]) * np.exp(np.pi / 2 * complex(0, 1))
        norm_vec = np.array([norm_vec_c.real, norm_vec_c.imag])

        at_vec = scale_factor * at(frame)
        att_vec = scale_factor * att_val * tan_vec   
        ant_vec = scale_factor * ant_val * norm_vec

        vecs.append(plt.quiver([xt_val], [yx_val], [vt_vec[0]], [vt_vec[1]], units='xy', scale = 1, scale_units='xy', angles='xy'))
        vecs.append(plt.quiver([xt_val], [yx_val], [at_vec[0]], [at_vec[1]], units='xy', scale = 1, scale_units='xy', angles='xy', color='g'))
        vecs.append(plt.quiver([xt_val], [yx_val], [att_vec[0]], [att_vec[1]], units='xy', scale = 1, scale_units='xy', angles='xy', color='r'))
        vecs.append(plt.quiver([xt_val], [yx_val], [ant_vec[0]], [ant_vec[1]], units='xy', scale = 1, scale_units='xy', angles='xy', color='b'))

        caps.append(ax.text(xt_val + at_vec[0], yx_val + at_vec[1], r'$\vec{a}$',  fontsize=10, color='green'))
        caps.append(ax.text(xt_val + att_vec[0], yx_val + att_vec[1], r'$\vec{a}_{\tau}$',  fontsize=10, color='red'))
        caps.append(ax.text(xt_val + vt_vec[0], yx_val + vt_vec[1], r'$\vec{v}$',  fontsize=10))
        caps.append(ax.text(xt_val + ant_vec[0], yx_val + ant_vec[1], r'$\vec{a}_{n}$',  fontsize=10, color='blue'))
        
        vecs[0].set_label(r'$[{:.2f}, {:.2f}]$ - vector of speed'.format(vt_vec[0], vt_vec[1]))
        vecs[1].set_label(r'$[{:.2f}, {:.2f}]$ - vector of tangential acceleration'.format(att_vec[0], att_vec[1]))
        vecs[2].set_label(r'$[{:.2f}, {:.2f}]$ - vector of normal acceleration'.format(ant_vec[0], ant_vec[1]))
        vecs[3].set_label(r'$[{:.2f}, {:.2f}]$ - vector of total acceleration'.format(at_vec[0], at_vec[1]))
        ax.legend()

        ln.set_data(xdata, ydata)
        return ln,

    ani = FuncAnimation(fig, update_yx, frames=np.linspace(-5, 5, 120),
                        init_func=init_yx, blit=True)


    if gif == True:
        ani.save('task1.gif', dpi=100, writer=PillowWriter(fps=60))
        plt.close('all')
    else:
        plt.show()

In [6]:
solve_task1(scale=1)

In [6]:
def c_rot(vectors, angle, origin = [0, 0]):
        for i in range(len(vectors)):
            vectors[i] = complex(vectors[i][0] - origin[0], vectors[i][1] - origin[1])
            vectors[i] *= np.exp(angle * complex(0, 1))
            vectors[i] = [vectors[i].real + origin[0], vectors[i].imag + origin[1]]
        return vectors

In [7]:
OA = OP = 25
AC = 20
w = 1

In [9]:
t = Symbol('t')
O = np.array([0, 0])
P = np.array([0, OP])
P1 = c_rot([P], np.pi / 3)[0]
A = OA * sp.Point2D([sp.cos(t), sp.sin(t)])
B = sp.Ellipse(A, 80, 80).intersection(sp.Line(sp.Point(P), sp.Point(P1)))[0]
C = 1 / 4 * (B - A) + A
Pabc = [[A[0], A[1]], [B[0], B[1]], [C[0], C[1]]]
Vabc = [[sp.diff(i[0]), sp.diff(i[1])] for i in Pabc]
Aabc = [[sp.diff(i[0]), sp.diff(i[1])] for i in Vabc]

In [18]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.axis("off")

xdata, ydata = [], []
ln, = plt.plot([], [], 'b-')

vectors = []
points  = []
titles  = []
lines   = []

def init_model():
    ax.set_xlim(-130, 130)
    ax.set_ylim(-130, 130)
    plt.title('Model')
    return ln,
    
def plot_poly(Bt):
    h = 10
    w = 20
    poly = pltp.Polygon(c_rot([[Bt[0] - w, Bt[1] - h],
                               [Bt[0] - w, Bt[1] + h],
                               [Bt[0] + w, Bt[1] + h], 
                               [Bt[0] + w, Bt[1] - h]], np.pi / 6, Bt), 
                        fill=False, edgecolor="blue", linewidth=0.5)
    ax.add_patch(poly)
    
def clear_screen():
    if len(ax.patches):
        ax.patches.pop()
    
    while len(points):
        for item in points[-1]:
            item.remove()
        points.pop()
    
    while len(lines):
        for item in lines[-1]:
            item.remove()
        lines.pop()
        
    while len(titles):
        titles[-1].remove()
        titles.pop()
        
    while len(vectors):
        vectors[-1].remove()
        vectors.pop()

def update_model(frame):
    PFabc = [[float(i[0].subs(t, frame).evalf()), float(i[1].subs(t, frame).evalf())] for i in Pabc]
    VFabc = [[float(i[0].subs(t, frame).evalf()), float(i[1].subs(t, frame).evalf())] for i in Vabc]            
    AFabc = [[float(i[0].subs(t, frame).evalf()), float(i[1].subs(t, frame).evalf())] for i in Aabc]
    
    Tabc = [VFabc[i] / np.linalg.norm(VFabc[i]) for i in range(3)]
    Nabc = [c_rot([Tabc[i]], angle = np.pi / 2)[0] for i in range(3)]
    
    ATabc = [Tabc[i] * (np.dot(AFabc[i], VFabc[i]) / np.linalg.norm(VFabc[i])) for i in range(3)]
    ANabc = [Nabc[i] * (np.sqrt(np.linalg.norm(AFabc[i])**2 - ATabc[i]**2)) for i in range(3)]
    
    Pf = P
    Of = O
    Cf = C
    
    
    clear_screen()    
    plot_poly(PFabc[1])
        
    points.extend([
        plt.plot(i[0], i[1], marker="o", markersize=3, markeredgecolor="black", markerfacecolor="black")
        for i in [PFabc[0], PFabc[1], PFabc[2], Of, Pf]
    ])
    
    lines.extend([
        plt.plot([Pf[0], PFabc[1][0]], [Pf[1], PFabc[1][1]], linestyle="dashed", linewidth=0.5, color="green"),
        plt.plot([Pf[0], Of[0]], [Pf[1], Of[1]], linestyle="dashed", linewidth=0.5, color="blue"),
        plt.plot([PFabc[0][0], PFabc[1][0]], [PFabc[0][1], PFabc[1][1]], linewidth=0.5, color="black"),
        plt.plot([Of[0], PFabc[0][0]], [Of[1], PFabc[0][1]], linewidth=0.5, color="black")
    ])
    
    titles.extend([
        ax.text(i[0][0], i[0][1] + 3, i[1], fontsize=10)
        for i in [[PFabc[0], 'A'], [PFabc[1], 'B'], [PFabc[2], 'C'], [Of, 'O'], [Pf, 'P']]
    ])
    
    vectors.extend([
        plt.quiver([i[0]], [i[1]], [j[0]], [j[1]], color="black", units='xy', scale = 1, scale_units='xy', angles='xy')
        for i, j in zip(PFabc, VFabc)
    ])
    
    vectors.extend([
        plt.quiver([i[0]], [i[1]], [j[0]], [j[1]], color="green", units='xy', scale = 1, scale_units='xy', angles='xy')
        for i, j in zip(PFabc, AFabc)
    ])
    
    vectors.extend([
        plt.quiver([i[0]], [i[1]], [j[0]], [j[1]], color="red", units='xy', scale = 1, scale_units='xy', angles='xy')
        for i, j in zip(PFabc, ATabc)
    ])
    
    vectors.extend([
        plt.quiver([i[0]], [i[1]], [j[0]], [j[1]], color="blue", units='xy', scale = 1, scale_units='xy', angles='xy')
        for i, j in zip([PFabc[0], PFabc[2]], [ANabc[0], ANabc[2]])
    ])
    
    return ln,

anim = FuncAnimation(fig, update_model, frames=np.linspace(0, 2 * np.pi, 120),
                        init_func=init_model, blit=True)

anim.save('task2.gif', dpi=300, writer=PillowWriter(fps=60))
plt.close('all')